In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras 
from tensorflow.keras import layers
import tensorflow as tf
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
y = train['label']
train = train.drop(['label'],axis =1)
print(train)

       pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
0           0       0       0       0       0       0       0       0       0   
1           0       0       0       0       0       0       0       0       0   
2           0       0       0       0       0       0       0       0       0   
3           0       0       0       0       0       0       0       0       0   
4           0       0       0       0       0       0       0       0       0   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
41995       0       0       0       0       0       0       0       0       0   
41996       0       0       0       0       0       0       0       0       0   
41997       0       0       0       0       0       0       0       0       0   
41998       0       0       0       0       0       0       0       0       0   
41999       0       0       0       0       0       0       0       0       0   

       pixel9  ...  pixel77

In [6]:
X = tf.convert_to_tensor(train)
y = tf.convert_to_tensor(y)
Xtest =tf.convert_to_tensor(test)
print(X)
print(y)

tf.Tensor(
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]], shape=(42000, 784), dtype=int64)
tf.Tensor([1 0 1 ... 7 6 9], shape=(42000,), dtype=int64)


In [7]:
unique, counts = np.unique(np.array(y), return_counts=True)#understand distribution of y_train so as to choose metric for classification
#As the distribution of y_train in this dataset is somewhat uniform we can use accuarcy as evaluation metric 
print (np.asarray((unique, counts)).T)

[[   0 4132]
 [   1 4684]
 [   2 4177]
 [   3 4351]
 [   4 4072]
 [   5 3795]
 [   6 4137]
 [   7 4401]
 [   8 4063]
 [   9 4188]]


In [18]:
X = tf.cast(X,float)
shape = tf.shape( X ) # get dynamic tensor shape
X = X/255.0
print(X)
Xtest = tf.cast(Xtest,float)
Xtest = X_test/255.0

tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(42000, 784), dtype=float32)


In [19]:
X_train, X_cv, y_train, y_cv = train_test_split(X.numpy(), y.numpy(), test_size=0.20, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.30, random_state=42)
X_train, X_cv= tf.constant(X_train), tf.constant(X_cv)
y_train, y_cv = tf.constant(y_train), tf.constant(y_cv)
X_test, y_test = tf.constant(X_test), tf.constant(y_test)
print(X_cv.shape)
print(X_train.shape)
print(X_test.shape)

(8400, 784)
(23520, 784)
(10080, 784)


In [20]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-2,decay_steps=100000,decay_rate=0.8)

In [17]:
initializer1 = tf.keras.initializers.GlorotNormal()#Glorot Normal initializer/Xavier initialization works better for sigmoid layers
initializer2 = tf.keras.initializers.HeNormal()#He initalizer works better for Relu layers
#In general if we add more sigmoid layers to our neural network then its accuracy decreases as number of sigmoid layers increase
#Instead use relu layers or tanh activations
#Create a deep neural network using keras library 
# We can use dropout and Batch Normalization also
#optimiser adam is best as compared to gradient descent or rms-prop 
dropouts = [0.09,0.1,0.15,0.18]
for d in dropouts:
    print("Dropout: ",d)
    model = tf.keras.Sequential([
    keras.layers.InputLayer(784),
    layers.Dense(512,activation ="relu" ,name ="first_hidden_layer" , kernel_initializer = initializer2),
    layers.BatchNormalization(),
    layers.Dropout(d),
    layers.Dense(200,activation ="relu" ,name ="second_hidden_layer" , kernel_initializer = initializer2),
    layers.BatchNormalization(),
    layers.Dropout(d),
    layers.Dense(128,activation ="relu" ,name ="third_hidden_layer" , kernel_initializer = initializer2),
    layers.BatchNormalization(),
    layers.Dropout(d),
    layers.Dense(10,activation ="softmax" ,name ="output_layer" , kernel_initializer = initializer1)])
    model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(), 
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"])
    model.fit(X_cv, y_cv, batch_size=32, epochs=10, verbose=1)

Dropout:  0.09
Epoch 1/10
263/263 [==============================] - 1s 3ms/step - loss: 0.4834 - accuracy: 0.8507
Epoch 2/10
263/263 [==============================] - 1s 3ms/step - loss: 0.2131 - accuracy: 0.9323
Epoch 3/10
263/263 [==============================] - 1s 3ms/step - loss: 0.1525 - accuracy: 0.9500
Epoch 4/10
263/263 [==============================] - 1s 3ms/step - loss: 0.1161 - accuracy: 0.9644
Epoch 5/10
263/263 [==============================] - 1s 3ms/step - loss: 0.1053 - accuracy: 0.9650
Epoch 6/10
263/263 [==============================] - 1s 3ms/step - loss: 0.0937 - accuracy: 0.9713
Epoch 7/10
263/263 [==============================] - 1s 3ms/step - loss: 0.0649 - accuracy: 0.9789
Epoch 8/10
263/263 [==============================] - 1s 3ms/step - loss: 0.0559 - accuracy: 0.9814
Epoch 9/10
263/263 [==============================] - 1s 3ms/step - loss: 0.0638 - accuracy: 0.9793
Epoch 10/10
263/263 [==============================] - 1s 3ms/step - loss: 0.0694 - a

In [21]:
d =0.1 
model = tf.keras.Sequential([
    keras.layers.InputLayer(784),
    layers.Dense(512,activation ="relu" ,name ="first_hidden_layer" , kernel_initializer = initializer2),
    layers.BatchNormalization(),
    layers.Dropout(d),
    layers.Dense(256,activation ="relu" ,name ="second_hidden_layer" , kernel_initializer = initializer2),
    layers.BatchNormalization(),
    layers.Dropout(d),
    layers.Dense(64,activation ="relu" ,name ="third_hidden_layer" , kernel_initializer = initializer2),
    layers.BatchNormalization(),
    layers.Dropout(d),
    layers.Dense(10,activation ="softmax" ,name ="output_layer" , kernel_initializer = initializer1)])

In [22]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 first_hidden_layer (Dense)  (None, 512)               401920    
                                                                 
 batch_normalization_54 (Bat  (None, 512)              2048      
 chNormalization)                                                
                                                                 
 dropout_54 (Dropout)        (None, 512)               0         
                                                                 
 second_hidden_layer (Dense)  (None, 256)              131328    
                                                                 
 batch_normalization_55 (Bat  (None, 256)              1024      
 chNormalization)                                                
                                                                 
 dropout_55 (Dropout)        (None, 256)             

In [23]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(), 
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=["accuracy"])

In [24]:
model.fit(X, y, batch_size=32, epochs=100, verbose=1)

Epoch 1/100
1313/1313 [==============================] - 4s 3ms/step - loss: 0.3539 - accuracy: 0.8899
Epoch 2/100
1313/1313 [==============================] - 4s 3ms/step - loss: 0.2386 - accuracy: 0.9257
Epoch 3/100
1313/1313 [==============================] - 4s 3ms/step - loss: 0.1981 - accuracy: 0.9376
Epoch 4/100
1313/1313 [==============================] - 4s 3ms/step - loss: 0.1784 - accuracy: 0.9448
Epoch 5/100
1313/1313 [==============================] - 4s 3ms/step - loss: 0.1636 - accuracy: 0.9480
Epoch 6/100
1313/1313 [==============================] - 4s 3ms/step - loss: 0.1487 - accuracy: 0.9539
Epoch 7/100
1313/1313 [==============================] - 4s 3ms/step - loss: 0.1425 - accuracy: 0.9558
Epoch 8/100
1313/1313 [==============================] - 3s 3ms/step - loss: 0.1334 - accuracy: 0.9586
Epoch 9/100
1313/1313 [==============================] - 3s 3ms/step - loss: 0.1245 - accuracy: 0.9609
Epoch 10/100
1313/1313 [==============================] - 4s 3ms/step - l

In [25]:
model.evaluate(X_test, y_test, batch_size=32, verbose=1)

315/315 [==============================] - 0s 1ms/step - loss: 0.0033 - accuracy: 0.9991


[0.0032670360524207354, 0.9991071224212646]